In [1]:
!pip -q install yfinance

In [0]:
import yfinance as yf
import pandas as pd
import numpy as np
import requests
from sklearn.preprocessing import MinMaxScaler
import bs4 as bs
from random import shuffle
import random
from collections import deque

In [0]:
tickers = []
companies = []
req = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = bs.BeautifulSoup(req.text)
table = soup.find('table',{'class':'wikitable sortable'})
for row in table.findAll('tr')[1:]:
  info = row.findAll('td')
  try:
    #if info[3].text == 'Information Technology':
    tickers.append(info[0].text.replace('\n',''))
    full=info[1].text
    companies.append(full)
  except: pass

In [0]:
def classify(current,future):
    pc = 100*(future-current)/current
    if abs(pc) > 1:
        if current<future:
            return 1
        return 0
def preprocess_df(df):
    global sequential_data
    for col in df.columns:
        if col != 'class':
            #scaler = MinMaxScaler()
            df[col] = df[col].pct_change()
            #df.dropna(inplace=True)
            #df[col] =  scaler.fit_transform(np.array(df[col]).reshape(-1,1))
            #joblib.dump(scaler, f'scaler_{col}.pkl')
    df.dropna(inplace=True)
    
    prev_days = deque(maxlen=60)
    for i in df.values:
        prev_days.append([n for n in i[:-1]])
        if len(prev_days) == 60:
            sequential_data.append([np.array(prev_days), i[-1]])

    random.shuffle(sequential_data)
    
    '''
    for seq, target in sequential_data:
        X.append(seq)
        y.append(target)
    '''
def rsi(price, n=14):
    ''' rsi indicator '''
    gain = (price-price.shift(1)).fillna(0) # calculate price gain with previous day, first row nan is filled with 0

    def rsiCalc(p):
        # subfunction for calculating rsi for one lookback period
        avgGain = p[p>0].sum()/n
        avgLoss = -p[p<0].sum()/n 
        rs = avgGain/avgLoss
        return 100 - 100/(1+rs)

    # run for all periods with rolling_apply
    return gain.rolling(n).apply(rsiCalc)         

def get_technical_indicators(dataset):
    # Create 7 and 21 days Moving Average
    dataset['ma7'] = dataset['Close'].rolling(window=7).mean()
    dataset['ma21'] = dataset['Close'].rolling(window=21).mean()
    
    # Create MACD
    dataset['26ema'] = dataset['Close'].ewm(span=26).mean()
    dataset['12ema'] = dataset['Close'].ewm(span=12).mean()
    dataset['MACD'] = (dataset['12ema']-dataset['26ema'])
    dataset['signal'] = dataset['MACD'].rolling(window=9).mean()
    dataset['MACD_signal'] = df[['MACD', 'signal']].apply(lambda row: (row.iloc[0]-row.iloc[1])/row.iloc[0], axis=1)

    # Create Bollinger Bands
    dataset['20sd'] = dataset['Close'].rolling(20).std()
    dataset['upper_band'] = dataset['ma21'] + (dataset['20sd']*2)
    dataset['lower_band'] = dataset['ma21'] - (dataset['20sd']*2)
    
    dataset['upper_band'] = df[['Close', 'upper_band']].apply(lambda row: (row.iloc[0]-row.iloc[1])/row.iloc[0], axis=1)
    dataset['lower_band'] = df[['Close', 'lower_band']].apply(lambda row: (row.iloc[0]-row.iloc[1])/row.iloc[0], axis=1)
    # Create Exponential moving average
    dataset['ema7'] = dataset['Close'].ewm(span=7,adjust=False).mean()
    dataset['ema21'] = dataset['Close'].ewm(span=21,adjust=False).mean()

    #RSI
    dataset['rsi'] = rsi(dataset['Close'])
    
    return dataset
#for file in glob('stock/^GSPC.csv'):


In [0]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [6]:
global sequential_data
sequential_data=[]
for ticker in tickers:
  try:
    print(ticker)
    stock = yf.Ticker(ticker)
    df = stock.history(period='3y')
    df['Volume'] =df['Volume'].replace(0,np.nan)
    df = get_technical_indicators(df)
    df.dropna(inplace=True)
    
    df['future'] = df['Close'].shift(-5)
    df = df.drop('Dividends',1)
    df['class'] = list(map(classify,df['Close'],df['future']))
    df = df.drop('future',1)
    df = df.drop('Stock Splits',1)
    df.fillna(method='ffill')
    df.dropna(inplace=True)
    preprocess_df(df)
    print(len(sequential_data))
  except Exception as e:
    print(e)

MMM
475
ABT
931
ABBV
1430
ABMD
1986
ACN
2424
ATVI
2939
ADBE
3439
AMD
4028
AAP
4546
AES
5021
AFL
5406
A
5906
APD
6330
AKAM
6837
ALK
7364
ALB
7901
ARE
8304
ALXN
8842
ALGN
9410
ALLE
9874
ADS
10414
LNT
10827
ALL
11230
GOOGL
11711
GOOG
12192
MO
12687
AMZN
13195
AMCR


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:55: RuntimeWarning: invalid value encountered in double_scalars


13440
AEE
13889
AAL
14441
AEP
14864
AXP
15313
AIG
15804
AMT
16258
AWK
16679
AMP
17204
ABC
17737
AME
18178
AMGN
18638
APH
19098
ADI
19607
ANSS
20096
ANTM
20592
AON
21019
AOS
21509
APA
22081
AIV
22499
AAPL
23016
AMAT
23574
APTV
24099
ADM
24543
ANET
25084
AJG
25472
AIZ
25920
T
26369
ATO
26750
ADSK
27275
ADP
27709
AZO
28210
AVB
28610
AVY
29096
BKR
29656
BLL
30136
BAC
30626
BK
31098
BAX
31546
BDX
31995
BRK.B
- BRK.B: No data found, symbol may be delisted
Wrong number of items passed 2, placement implies 1
BBY
32537
BIIB
33023
BLK
33487
BA
33965
BKNG
34464
BWA
34989
BXP
35420
BSX
35879
BMY
36341
AVGO
36878
BR
37349
BF.B
- BF.B: No data found for this date range, symbol may be delisted
Wrong number of items passed 2, placement implies 1
CHRW
37822
COG
38347
CDNS
38847
CPB
39295
COF
39805
CAH
40312
KMX
40827
CCL
41339
CARR
41339
CAT
41852
CBOE
42318
CBRE
42787
CDW
43284
CE
43738
CNC
44257
CNP
44694
CTL
45244
CERN
45697
CF
46231
SCHW
46731
CHTR
47226
CVX
47693
CMG
48197
CB
48619
CHD
49062
CI
49

In [0]:
import pickle

with open('seq.pkl','wb') as f:
  pickle.dump(sequential_data,f)
